In [1]:
import torch

torch.load("/home/hrc/gsoc/batch_1_0/data21000_s14.pt")

Data(x=[316, 6], edge_index=[2, 6634], edge_attr=[6634, 4], y=[6634])

In [2]:
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

# data_list = [Data(...), ..., Data(...)]
# loader = DataLoader(data_list, batch_size=32)

In [21]:
from pathlib import Path

from torch.utils.data import Dataset  # not the one from PyG!
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


import torch


class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()
        self.graphs = list(path.glob("**/*.pt"))
    
    def __getitem__(self, idx):
        return torch.load(self.graphs[idx])
    
    def __len__(self) -> int:
        return len(self.graphs)


dataset = MyDataset(Path("/home/hrc/gsoc/"))
loader = DataLoader(dataset, batch_size=32)

In [34]:
batch = next(iter(loader))

In [35]:
batch

DataBatch(x=[9569, 6], edge_index=[2, 116642], edge_attr=[116642, 4], y=[116642], batch=[9569], ptr=[33])

In [36]:
count = 0
for batch in loader:
    print(batch)
    count+=1

DataBatch(x=[9569, 6], edge_index=[2, 116642], edge_attr=[116642, 4], y=[116642], batch=[9569], ptr=[33])
DataBatch(x=[10036, 6], edge_index=[2, 127298], edge_attr=[127298, 4], y=[127298], batch=[10036], ptr=[33])
DataBatch(x=[9840, 6], edge_index=[2, 121064], edge_attr=[121064, 4], y=[121064], batch=[9840], ptr=[33])
DataBatch(x=[9770, 6], edge_index=[2, 124730], edge_attr=[124730, 4], y=[124730], batch=[9770], ptr=[33])
DataBatch(x=[9931, 6], edge_index=[2, 127558], edge_attr=[127558, 4], y=[127558], batch=[9931], ptr=[33])
DataBatch(x=[9869, 6], edge_index=[2, 139878], edge_attr=[139878, 4], y=[139878], batch=[9869], ptr=[33])
DataBatch(x=[9688, 6], edge_index=[2, 118646], edge_attr=[118646, 4], y=[118646], batch=[9688], ptr=[33])
DataBatch(x=[9827, 6], edge_index=[2, 134194], edge_attr=[134194, 4], y=[134194], batch=[9827], ptr=[33])
DataBatch(x=[9712, 6], edge_index=[2, 129030], edge_attr=[129030, 4], y=[129030], batch=[9712], ptr=[33])
DataBatch(x=[10013, 6], edge_index=[2, 13444

DataBatch(x=[9610, 6], edge_index=[2, 118564], edge_attr=[118564, 4], y=[118564], batch=[9610], ptr=[33])
DataBatch(x=[9259, 6], edge_index=[2, 112460], edge_attr=[112460, 4], y=[112460], batch=[9259], ptr=[33])
DataBatch(x=[9116, 6], edge_index=[2, 113936], edge_attr=[113936, 4], y=[113936], batch=[9116], ptr=[33])
DataBatch(x=[9213, 6], edge_index=[2, 111814], edge_attr=[111814, 4], y=[111814], batch=[9213], ptr=[33])
DataBatch(x=[8887, 6], edge_index=[2, 115462], edge_attr=[115462, 4], y=[115462], batch=[8887], ptr=[33])
DataBatch(x=[9641, 6], edge_index=[2, 126036], edge_attr=[126036, 4], y=[126036], batch=[9641], ptr=[33])
DataBatch(x=[9756, 6], edge_index=[2, 137624], edge_attr=[137624, 4], y=[137624], batch=[9756], ptr=[33])
DataBatch(x=[9747, 6], edge_index=[2, 144160], edge_attr=[144160, 4], y=[144160], batch=[9747], ptr=[33])
DataBatch(x=[9356, 6], edge_index=[2, 128544], edge_attr=[128544, 4], y=[128544], batch=[9356], ptr=[33])
DataBatch(x=[10017, 6], edge_index=[2, 123688]

DataBatch(x=[9182, 6], edge_index=[2, 124430], edge_attr=[124430, 4], y=[124430], batch=[9182], ptr=[33])
DataBatch(x=[8519, 6], edge_index=[2, 109696], edge_attr=[109696, 4], y=[109696], batch=[8519], ptr=[33])
DataBatch(x=[9682, 6], edge_index=[2, 126156], edge_attr=[126156, 4], y=[126156], batch=[9682], ptr=[33])
DataBatch(x=[9239, 6], edge_index=[2, 116990], edge_attr=[116990, 4], y=[116990], batch=[9239], ptr=[33])
DataBatch(x=[9755, 6], edge_index=[2, 127212], edge_attr=[127212, 4], y=[127212], batch=[9755], ptr=[33])
DataBatch(x=[9945, 6], edge_index=[2, 128804], edge_attr=[128804, 4], y=[128804], batch=[9945], ptr=[33])
DataBatch(x=[9510, 6], edge_index=[2, 130578], edge_attr=[130578, 4], y=[130578], batch=[9510], ptr=[33])
DataBatch(x=[9780, 6], edge_index=[2, 130056], edge_attr=[130056, 4], y=[130056], batch=[9780], ptr=[33])
DataBatch(x=[9690, 6], edge_index=[2, 126624], edge_attr=[126624, 4], y=[126624], batch=[9690], ptr=[33])
DataBatch(x=[8853, 6], edge_index=[2, 118498],

DataBatch(x=[9565, 6], edge_index=[2, 119980], edge_attr=[119980, 4], y=[119980], batch=[9565], ptr=[33])
DataBatch(x=[10074, 6], edge_index=[2, 135932], edge_attr=[135932, 4], y=[135932], batch=[10074], ptr=[33])
DataBatch(x=[9693, 6], edge_index=[2, 123270], edge_attr=[123270, 4], y=[123270], batch=[9693], ptr=[33])
DataBatch(x=[9623, 6], edge_index=[2, 122470], edge_attr=[122470, 4], y=[122470], batch=[9623], ptr=[33])
DataBatch(x=[9132, 6], edge_index=[2, 122326], edge_attr=[122326, 4], y=[122326], batch=[9132], ptr=[33])
DataBatch(x=[9618, 6], edge_index=[2, 120322], edge_attr=[120322, 4], y=[120322], batch=[9618], ptr=[33])
DataBatch(x=[9575, 6], edge_index=[2, 118214], edge_attr=[118214, 4], y=[118214], batch=[9575], ptr=[33])
DataBatch(x=[8934, 6], edge_index=[2, 110350], edge_attr=[110350, 4], y=[110350], batch=[8934], ptr=[33])
DataBatch(x=[9169, 6], edge_index=[2, 109782], edge_attr=[109782, 4], y=[109782], batch=[9169], ptr=[33])
DataBatch(x=[10242, 6], edge_index=[2, 13734

In [37]:
count

313

In [38]:
# should be 313 * 32 + 12 and it is! 
len(dataset)

9996

In [40]:
data = torch.load("/home/hrc/gsoc/batch_1_0/data21000_s14.pt")
data

Data(x=[316, 6], edge_index=[2, 6634], edge_attr=[6634, 4], y=[6634])

In [5]:
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([316, 6])
tensor([[  0.0684,  -0.9035,  -1.4980,  -3.7805, -13.9569,  -4.3680],
        [  0.0686,  -0.9035,  -1.5020,  -3.7805, -13.9203,  -4.3556],
        [  0.1340,  -0.8955,  -1.4980,  -3.1095,  -7.0661,  -2.4080],
        ...,
        [  0.1050,  -0.9137,   1.5020,   3.3548,  -9.1744,  -2.5514],
        [  0.1128,  -0.8964,   1.4980,   3.2808,  -8.3998,  -2.8340],
        [  0.1047,  -0.9137,   1.4980,   3.3548,  -9.1993,  -2.5569]])


In [6]:
print(f'edge_index = {data.edge_index.shape}')
print(data.edge_index)

edge_index = torch.Size([2, 6634])
tensor([[  0,   0,   1,  ..., 313, 315, 314],
        [  3,   4,   3,  ..., 311, 311, 312]])


In [7]:
print(f'edge_attributes = {data.edge_attr.shape}')
print(data.edge_attr)

edge_attributes = torch.Size([6634, 4])
tensor([[-0.0092, -0.0007,  0.2000,  0.0025],
        [-0.0090, -0.0007,  0.1960,  0.0022],
        [-0.0094, -0.0007,  0.2040,  0.0024],
        ...,
        [-0.0140,  0.0012, -0.2040,  0.0045],
        [-0.0138,  0.0012, -0.2000,  0.0046],
        [-0.0149, -0.0011, -0.1960,  0.0038]])


In [8]:
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([6634])
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [9]:
from torch_geometric.utils import to_dense_adj

A = to_dense_adj(data.edge_index)[0].numpy().astype(int)
print(f'A = {A.shape}')
print(A)

A = (316, 316)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([6634])
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [11]:
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')


Edges are directed: True
Graph has isolated nodes: False
Graph has loops: False


In [12]:
# Finding edge embeddings

# x = ...  # Node feature matrix: [num_nodes, num_features]
# edge_index = ... # Edge indices: [2, num_edges]

src, dst = data.edge_index
score = (data.x[src] * data.x[dst]).sum(dim=-1)

loss = F.cross_entropy(score, data.y)  # Classification

In [13]:
score

tensor([264.1794, 263.3403, 263.5314,  ..., 118.7073, 118.9811, 104.0850])

In [14]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Encoder(torch.nn.Module):
    def __init__(self, in_channel, hidden_dim):
        super(Encoder, self).__init__()
        self.conv1 = GCNConv(in_channel, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        
        return x

class Decoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels):
        super(Decoder, self).__init__()
        self.fc1 = torch.nn.Linear(in_channels, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, out_channels)

    def forward(self, x, edge_index):
        x = torch.cat([x[edge_index[0]], x[edge_index[1]]], dim=-1)
        print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class EdgeClassifier(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(EdgeClassifier, self).__init__()
        self.encoder = Encoder(in_channels, 32)
        self.decoder = Decoder(32 * 2, 64, out_channels)

    def forward(self, x, edge_index):
        x = self.encoder(x, edge_index)
        x = self.decoder(x, edge_index)
        return x


In [50]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def train_model():
    model.train()
    
    loss_all = 0
    for graph_data in loader:
        optimizer.zero_grad()
        output = model(graph_data.x, graph_data.edge_index)
        print(output)
        loss = criterion(output, graph_data.y.to(torch.long))
        loss.backward()
        loss_all += loss
        optimizer.step()


In [51]:
model = EdgeClassifier(in_channels=6, out_channels=2)#.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Define the loss function
criterion = torch.nn.CrossEntropyLoss()

loader = DataLoader(dataset, batch_size=32)#.to(device)

# Train the model
for epoch in range(5):
    train_model()

torch.Size([116642, 64])
tensor([[ 0.2749, -0.4380],
        [ 0.2641, -0.2864],
        [ 0.2927, -0.4400],
        ...,
        [ 0.2501, -0.5813],
        [ 0.1600, -0.3121],
        [ 0.1589, -0.3119]], grad_fn=<AddmmBackward0>)
torch.Size([127298, 64])
tensor([[ 3.0920, -3.0600],
        [ 3.0922, -3.0601],
        [ 2.3965, -2.3629],
        ...,
        [ 2.7158, -2.9523],
        [ 1.4089, -1.4930],
        [ 2.9359, -3.1421]], grad_fn=<AddmmBackward0>)
torch.Size([121064, 64])
tensor([[ 1.1778, -0.9153],
        [ 1.1778, -0.9153],
        [ 1.1629, -0.9332],
        ...,
        [ 0.4613, -0.9767],
        [ 0.4612, -0.9766],
        [ 0.4260, -0.8914]], grad_fn=<AddmmBackward0>)
torch.Size([124730, 64])
tensor([[ 0.7033, -0.9025],
        [ 0.7033, -0.9025],
        [ 0.6324, -0.7885],
        ...,
        [ 0.2001, -0.2800],
        [ 0.2740, -0.5010],
        [ 0.2740, -0.5009]], grad_fn=<AddmmBackward0>)
torch.Size([127558, 64])
tensor([[ 0.7537, -1.0124],
        [ 0.753

torch.Size([117866, 64])
tensor([[ 0.1248, -0.3533],
        [ 0.2091, -0.4686],
        [ 0.2091, -0.4684],
        ...,
        [-1.1603,  0.1887],
        [-1.1605,  0.1885],
        [-1.0321,  0.1134]], grad_fn=<AddmmBackward0>)
torch.Size([121454, 64])
tensor([[ 0.1067, -0.7789],
        [ 0.1067, -0.7789],
        [ 0.1067, -0.7788],
        ...,
        [-0.8550, -0.3354],
        [-0.8378, -0.1686],
        [-0.8379, -0.1684]], grad_fn=<AddmmBackward0>)
torch.Size([126642, 64])
tensor([[-0.2737, -0.5158],
        [-0.2405, -0.4303],
        [-0.2738, -0.5157],
        ...,
        [-1.2419,  0.0808],
        [-1.2419,  0.0809],
        [-1.2419,  0.0809]], grad_fn=<AddmmBackward0>)
torch.Size([124242, 64])
tensor([[-0.1434, -0.2513],
        [-0.1434, -0.2513],
        [-0.2428, -0.1586],
        ...,
        [-1.0701,  0.2383],
        [-1.3590,  0.3194],
        [-0.8543,  0.1623]], grad_fn=<AddmmBackward0>)
torch.Size([125986, 64])
tensor([[-0.2723, -0.4048],
        [-0.272

torch.Size([122238, 64])
tensor([[-0.2889, -0.0260],
        [-0.2717, -0.0703],
        [-0.3368, -0.0347],
        ...,
        [-2.0220,  0.5091],
        [-2.0221,  0.5091],
        [-1.6295,  0.4198]], grad_fn=<AddmmBackward0>)
torch.Size([112468, 64])
tensor([[-3.7764e-01, -6.1226e-02],
        [-3.7762e-01, -6.1249e-02],
        [-3.5217e-01,  1.6578e-03],
        ...,
        [-2.0696e+00,  6.6490e-01],
        [-2.1682e+00,  7.3005e-01],
        [-2.1681e+00,  7.3006e-01]], grad_fn=<AddmmBackward0>)
torch.Size([122348, 64])
tensor([[-0.3155, -0.0604],
        [-0.3155, -0.0604],
        [-0.3155, -0.0604],
        ...,
        [-1.8753,  0.5829],
        [-2.0605,  0.6378],
        [-2.0604,  0.6378]], grad_fn=<AddmmBackward0>)
torch.Size([117172, 64])
tensor([[-0.4851,  0.0828],
        [-0.4235,  0.0497],
        [-0.4281,  0.0696],
        ...,
        [-2.0651,  0.6180],
        [-2.0652,  0.6180],
        [-2.0493,  0.5951]], grad_fn=<AddmmBackward0>)
torch.Size([120780, 

torch.Size([132598, 64])
tensor([[-0.5026, -0.2045],
        [-0.5027, -0.2044],
        [-0.5026, -0.2046],
        ...,
        [-1.7686,  0.4207],
        [-1.7687,  0.4207],
        [-1.8809,  0.3970]], grad_fn=<AddmmBackward0>)
torch.Size([138266, 64])
tensor([[-0.0694, -0.0571],
        [-0.2225, -0.1433],
        [-0.1252, -0.0528],
        ...,
        [-2.0734,  0.7498],
        [-1.9359,  0.5892],
        [-1.9358,  0.5892]], grad_fn=<AddmmBackward0>)
torch.Size([117178, 64])
tensor([[-0.2663, -0.1441],
        [-0.2662, -0.1441],
        [-0.2746, -0.2025],
        ...,
        [-1.7730,  0.3924],
        [-1.8225,  0.4490],
        [-1.7870,  0.4545]], grad_fn=<AddmmBackward0>)
torch.Size([137778, 64])
tensor([[-0.2703, -0.2101],
        [-0.2703, -0.2102],
        [-0.3067, -0.1728],
        ...,
        [-1.8897,  0.6924],
        [-1.8580,  0.6475],
        [-1.8581,  0.6474]], grad_fn=<AddmmBackward0>)
torch.Size([112668, 64])
tensor([[-0.1329, -0.1401],
        [-0.132

tensor([[-0.0945, -0.2038],
        [-0.0945, -0.2038],
        [-0.1664, -0.1593],
        ...,
        [-1.9064,  0.2090],
        [-2.2663,  0.2703],
        [-2.2661,  0.2704]], grad_fn=<AddmmBackward0>)
torch.Size([130860, 64])
tensor([[-0.3997, -0.1543],
        [-0.3997, -0.1544],
        [-0.3536, -0.1893],
        ...,
        [-1.5581,  0.5315],
        [-1.5583,  0.5318],
        [-1.3956,  0.2900]], grad_fn=<AddmmBackward0>)
torch.Size([125230, 64])
tensor([[-0.3841,  0.0220],
        [-0.3841,  0.0220],
        [-0.3841,  0.0221],
        ...,
        [-2.0271,  0.2629],
        [-2.0590,  0.2768],
        [-1.7927,  0.1852]], grad_fn=<AddmmBackward0>)
torch.Size([132446, 64])
tensor([[-0.4687, -0.1531],
        [-0.4686, -0.1532],
        [-0.4173, -0.1336],
        ...,
        [-1.8277,  0.1865],
        [-1.6693,  0.1612],
        [-2.0762,  0.2823]], grad_fn=<AddmmBackward0>)
torch.Size([122010, 64])
tensor([[-0.4011, -0.1671],
        [-0.4012, -0.1671],
        [-0.

...

torch.Size([127024, 64])
tensor([[-1.8342,  0.4827],
        [-1.8341,  0.4826],
        [-1.7815,  0.6264],
        ...,
        [-1.7973,  1.2988],
        [-1.1783,  0.7349],
        [-1.2110,  0.7506]], grad_fn=<AddmmBackward0>)
torch.Size([131610, 64])
tensor([[-1.1806,  0.1606],
        [-1.1799,  0.1599],
        [-2.2410,  0.9174],
        ...,
        [-1.4752,  0.8733],
        [-1.2845,  0.7964],
        [-1.4596,  0.8604]], grad_fn=<AddmmBackward0>)
torch.Size([116116, 64])
tensor([[-1.8054,  0.3403],
        [-1.8057,  0.3407],
        [-1.8920,  0.7439],
        ...,
        [-1.3860,  0.8283],
        [-1.3860,  0.8283],
        [-1.4830,  0.8766]], grad_fn=<AddmmBackward0>)
torch.Size([132840, 64])
tensor([[-1.5161,  0.3215],
        [-2.1158,  0.6425],
        [-2.1163,  0.6442],
        ...,
        [-1.7362,  1.3089],
        [-1.5710,  0.9746],
        [-1.6133,  0.9490]], grad_fn=<AddmmBackward0>)
torch.Size([140970, 64])
tensor([[-2.8939,  1.4747],
        [-2.894

torch.Size([121746, 64])
tensor([[-2.1471,  0.9175],
        [-2.1478,  0.9182],
        [-2.1471,  0.9173],
        ...,
        [-1.4019,  0.5824],
        [-1.4513,  0.6160],
        [-1.4513,  0.6160]], grad_fn=<AddmmBackward0>)
torch.Size([111834, 64])
tensor([[-1.1417, -0.2228],
        [-1.1411, -0.2234],
        [-0.9618, -0.2695],
        ...,
        [-1.3832,  0.6061],
        [-1.7417,  0.8549],
        [-1.7416,  0.8549]], grad_fn=<AddmmBackward0>)
torch.Size([126410, 64])
tensor([[-1.4492,  0.1984],
        [-1.4490,  0.1982],
        [-1.4491,  0.1983],
        ...,
        [-1.3678,  0.5624],
        [-1.3678,  0.5623],
        [-1.3677,  0.5624]], grad_fn=<AddmmBackward0>)
torch.Size([119742, 64])
tensor([[-2.2043,  1.0129],
        [-1.7711,  0.7512],
        [-1.7708,  0.7508],
        ...,
        [-1.3596,  0.5009],
        [-1.5694,  0.6622],
        [-1.5694,  0.6621]], grad_fn=<AddmmBackward0>)
torch.Size([127852, 64])
tensor([[-1.9124,  0.7716],
        [-1.156

torch.Size([127898, 64])
tensor([[-1.9339,  0.1901],
        [-1.9340,  0.1901],
        [-2.8537,  1.2454],
        ...,
        [-2.0084,  0.9847],
        [-2.0258,  1.2558],
        [-1.8875,  0.9443]], grad_fn=<AddmmBackward0>)
torch.Size([125154, 64])
tensor([[-1.7299,  0.4006],
        [-1.7304,  0.4012],
        [-1.6338,  0.1441],
        ...,
        [-1.5520,  0.6387],
        [-2.3062,  1.0289],
        [-2.3060,  1.0289]], grad_fn=<AddmmBackward0>)
torch.Size([136290, 64])
tensor([[-1.3845,  0.3136],
        [-1.3839,  0.3129],
        [-1.3865,  0.3160],
        ...,
        [-1.7931,  0.9144],
        [-1.7931,  0.9144],
        [-1.7931,  0.9144]], grad_fn=<AddmmBackward0>)
torch.Size([145042, 64])
tensor([[-2.4792,  0.7911],
        [-2.3970,  0.6309],
        [-2.3972,  0.6310],
        ...,
        [-2.3049,  1.2852],
        [-2.0781,  1.2315],
        [-1.2576,  0.6215]], grad_fn=<AddmmBackward0>)
torch.Size([128736, 64])
tensor([[-1.6696,  0.6831],
        [-1.670

torch.Size([135968, 64])
tensor([[-2.5713,  0.8675],
        [-2.8217,  1.6560],
        [-2.7448,  1.4057],
        ...,
        [-1.2437,  0.7209],
        [-1.7263,  1.2292],
        [-1.7262,  1.2292]], grad_fn=<AddmmBackward0>)
torch.Size([120716, 64])
tensor([[-2.9015,  1.4621],
        [-2.8452,  0.9468],
        [-2.8455,  0.9470],
        ...,
        [-1.1451,  0.5930],
        [-1.0832,  0.5654],
        [-1.0831,  0.5654]], grad_fn=<AddmmBackward0>)
torch.Size([133548, 64])
tensor([[-1.4760,  0.2581],
        [-1.4752,  0.2574],
        [-1.4760,  0.2587],
        ...,
        [-0.9590,  0.4420],
        [-1.1052,  0.5243],
        [-1.1850,  0.6951]], grad_fn=<AddmmBackward0>)
torch.Size([140166, 64])
tensor([[-1.7086,  0.1237],
        [-1.4115, -0.1213],
        [-1.9022,  0.9885],
        ...,
        [-1.3556,  0.8487],
        [-0.9569,  0.4911],
        [-0.9568,  0.4910]], grad_fn=<AddmmBackward0>)
torch.Size([109998, 64])
tensor([[-1.5772,  0.0696],
        [-1.577

torch.Size([133534, 64])
tensor([[-2.0010,  0.7995],
        [-2.0019,  0.8004],
        [-2.3086,  0.9571],
        ...,
        [-1.1918,  0.5450],
        [-1.1917,  0.5448],
        [-1.4394,  0.8558]], grad_fn=<AddmmBackward0>)
torch.Size([115584, 64])
tensor([[-2.0137,  0.9511],
        [-1.5761,  0.0608],
        [-1.7499,  0.2317],
        ...,
        [-1.2463,  0.6279],
        [-0.9854,  0.4980],
        [-0.9855,  0.4982]], grad_fn=<AddmmBackward0>)
torch.Size([134534, 64])
tensor([[-0.9140, -0.7285],
        [-0.9148, -0.7279],
        [-2.3298,  0.8829],
        ...,
        [-0.9661,  0.4944],
        [-1.0331,  0.5110],
        [-1.4128,  0.8471]], grad_fn=<AddmmBackward0>)
torch.Size([128578, 64])
tensor([[-2.9033,  1.4608],
        [-2.9036,  1.4609],
        [-2.3716,  0.9682],
        ...,
        [-1.1243,  0.6394],
        [-1.1244,  0.6394],
        [-1.4468,  1.0020]], grad_fn=<AddmmBackward0>)
torch.Size([104652, 64])
tensor([[-1.9344,  0.7811],
        [-1.935

In [52]:
# Test the model
model.eval()
with torch.no_grad():
    for graph_data in loader:
        output = model(graph_data.x, graph_data.edge_index)
        pred = output.argmax(dim=1)
        correct = pred.eq(graph_data.y).sum().item()
        accuracy = correct / graph_data.y.size(0)
        print('Accuracy: {:.4f}'.format(accuracy))

torch.Size([116642, 64])
Accuracy: 0.9170
torch.Size([127298, 64])
Accuracy: 0.9153
torch.Size([121064, 64])
Accuracy: 0.9144
torch.Size([124730, 64])
Accuracy: 0.9175
torch.Size([127558, 64])
Accuracy: 0.9171
torch.Size([139878, 64])
Accuracy: 0.9222
torch.Size([118646, 64])
Accuracy: 0.9148
torch.Size([134194, 64])
Accuracy: 0.9189
torch.Size([129030, 64])
Accuracy: 0.9153
torch.Size([134446, 64])
Accuracy: 0.9189
torch.Size([130848, 64])
Accuracy: 0.9225
torch.Size([123130, 64])
Accuracy: 0.9150
torch.Size([135848, 64])
Accuracy: 0.9177
torch.Size([108688, 64])
Accuracy: 0.9151
torch.Size([134728, 64])
Accuracy: 0.9204
torch.Size([113464, 64])
Accuracy: 0.9122
torch.Size([136756, 64])
Accuracy: 0.9212
torch.Size([111750, 64])
Accuracy: 0.9145
torch.Size([125452, 64])
Accuracy: 0.9188
torch.Size([135226, 64])
Accuracy: 0.9209
torch.Size([110908, 64])
Accuracy: 0.9136
torch.Size([119316, 64])
Accuracy: 0.9170
torch.Size([126074, 64])
Accuracy: 0.9163
torch.Size([95366, 64])
Accuracy: 

torch.Size([123176, 64])
Accuracy: 0.9184
torch.Size([133562, 64])
Accuracy: 0.9188
torch.Size([125432, 64])
Accuracy: 0.9185
torch.Size([132940, 64])
Accuracy: 0.9221
torch.Size([123608, 64])
Accuracy: 0.9226
torch.Size([127958, 64])
Accuracy: 0.9177
torch.Size([132284, 64])
Accuracy: 0.9150
torch.Size([121188, 64])
Accuracy: 0.9192
torch.Size([138424, 64])
Accuracy: 0.9213
torch.Size([142176, 64])
Accuracy: 0.9186
torch.Size([118788, 64])
Accuracy: 0.9121
torch.Size([140274, 64])
Accuracy: 0.9229
torch.Size([140624, 64])
Accuracy: 0.9207
torch.Size([114720, 64])
Accuracy: 0.9147
torch.Size([125866, 64])
Accuracy: 0.9195
torch.Size([119998, 64])
Accuracy: 0.9203
torch.Size([145842, 64])
Accuracy: 0.9261
torch.Size([113694, 64])
Accuracy: 0.9133
torch.Size([126216, 64])
Accuracy: 0.9156
torch.Size([127974, 64])
Accuracy: 0.9122
torch.Size([126634, 64])
Accuracy: 0.9133
torch.Size([126566, 64])
Accuracy: 0.9174
torch.Size([120256, 64])
Accuracy: 0.9108
torch.Size([137520, 64])
Accuracy: